<div style="font-size: 20px; font-weight: 600;text-transform:uppercase; color:#23547b; background:#B3E5FC;text-align:center">
<h1 > BBM 409: Introduction to Machine Learning Lab. </h1>
<h1> Assignment - 3 </h1>

</div>


<div style="text-align:center"> 
<h2 > Numan Kafadar 21946242</h2>
<h2> Umut Güngör 21946198</h2>
</div>
<h1  style="color:#805b36 ;background:#FFD8B2"> Importing Necessary Libraries and Reading Dataset</h1>

In [106]:
import numpy as np  
import pandas as pd
import math
import time
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfTransformer

In [107]:
#exploring the dataset
df = pd.read_csv("English_Dataset.csv")

#shuffle the dataset
df = df.sample(frac=1)

df.head(10)

,ArticleId,Text,Category
1341,312,lacroix label bought by us firm luxury goods g...,business
387,26,moya fights back for indian title carlos moya ...,sport
253,1387,adriano s chelsea link rejected adriano s agen...,sport
1329,1651,brown s poll campaign move denied the governme...,politics
950,30,halloween writer debra hill dies screenwriter ...,entertainment
245,1376,liberian economy starts to grow the liberian e...,business
649,1984,officials respond in court row australian tenn...,sport
1011,1081,outkast win at mtv europe awards us hip-hop du...,entertainment
1081,1925,straw to attend auschwitz service foreign secr...,politics
1380,208,blair to face trust issue head on tony blair s...,politics


In [108]:
#all the categories and their counts on dataset
print(df["Category"].value_counts())

sport            346
business         336
politics         274
entertainment    273
tech             261
Name: Category, dtype: int64


If we look at the counts of each category, we can say that our dataset is quite balanced. 

<h1  style="color:#805b36 ;background:#FFD8B2"> Splitting Data into Train and Test Samples , Defining Necessary Global Variables</h1>

Empirical studies show that the best results are obtained if we use 20-30% of the data for testing, and the remaining 70-80% of the data for training. So we will use %70 of our data as training set and remaining %30 of our data for testing.

In [109]:
#split data into train and test samples
X = df["Text"].to_numpy()
y = df["Category"].to_numpy()

#%70 training, %30 testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

#label_values
label_values = df["Category"].unique()
#ngram parameters
ngrams = { "unigram":(1,1), "bigram": (2,2)} 

<h1 style="font-size: 40px; font-weight: 600;text-transform:uppercase; color:#256029; background:#C8E6C9"> Part 1:  Understanding the data </h1>


In [110]:
#get occurences words list of each category
def most_occurence_values(df, labels, unmeaningful_words=set()):
    occurence_dict={}
    #for each category
    for l in labels:
        data = df[df["Category"]==l]
        word_counts = {}
        total_count = 0
        #for each text in category 
        for text in data["Text"].to_numpy():
            word_list = text.split()
            #for each word in text
            for word in word_list:
                #increase the count of word 
                if word in unmeaningful_words:
                    continue
                elif word in word_counts:
                    word_counts[word] += 1
                    total_count += 1
                else:
                    word_counts[word] = 1
                    total_count += 1
        #sort words list according to their occurence count
        word_counts = dict(sorted(word_counts.items(), key=lambda item: item[1], reverse = True))
        #set occurence counts to occurence frequency
        for word in word_counts:
            word_counts[word] = word_counts[word]/total_count
        
        occurence_dict[l] = word_counts
    return occurence_dict

#print first n words that has highest occurence frequency
def get_n_occurence_values(df,labels, n=3, unmeaningful_words=set()):
    #occurence dictionary
    occurences = most_occurence_values(df,label_values, unmeaningful_words)
    #for each category
    for l in occurences:
        #words and their frequencies as array
        word_list = list(occurences[l].keys())
        word_list_freqs = list(occurences[l].values())
        print("For {} label , most occurence {} words are :".format(l,n))
        for i in range(n):
            print("{:<12}  {:>12}".format(word_list[i],("frequency : %" +str(word_list_freqs[i]*100))))
        print()

In [111]:
get_n_occurence_values(df,label_values, 3)

For business label , most occurence 3 words are :
the           frequency : %6.349248759807982
to            frequency : %2.932820334695986
of            frequency : %2.548071356685459

For sport label , most occurence 3 words are :
the           frequency : %5.69680255106438
to            frequency : %2.7346375937257608
a             frequency : %2.2813065586486254

For politics label , most occurence 3 words are :
the           frequency : %6.452136509353569
to            frequency : %3.1700685793125833
of            frequency : %2.3024794059164875

For entertainment label , most occurence 3 words are :
the           frequency : %6.375743215077119
and           frequency : %2.325632418438316
to            frequency : %2.305886482810066

For tech label , most occurence 3 words are :
the           frequency : %5.718975455204794
to            frequency : %3.125548726953468
of            frequency : %2.603351528801008



As we can see above, words that has highest occurence frequency are a bit useless for analyzing data. Because these words are the common words like prepositions,conjunctions and pronouns in English language that has no effects on specifying category of the text. Alternatively, we can discard these words and do the same process again like below.

In [112]:
#words that has no meanings at all
my_set = ("-", "s", "t", "m", "o", "mr","said", "just","new")       # adding these words into stop words
unmeaningful_words = set(ENGLISH_STOP_WORDS).union(my_set)

#occurences without unnecessary words
get_n_occurence_values(df,label_values, 3,unmeaningful_words)

For business label , most occurence 3 words are :
year          frequency : %0.4980621753112888
firm          frequency : %0.3991094252494434
company       frequency : %0.39581100024738186

For sport label , most occurence 3 words are :
england       frequency : %0.533546979408155
game          frequency : %0.48581753716078
win           frequency : %0.4449065866630302

For politics label , most occurence 3 words are :
labour        frequency : %0.7601050209069398
government    frequency : %0.6968979935820556
blair         frequency : %0.6028977990988946

For entertainment label , most occurence 3 words are :
film          frequency : %1.0405313701700631
best          frequency : %0.8307799872504061
music         frequency : %0.47708157683686686

For tech label , most occurence 3 words are :
people        frequency : %0.9334190961990098
mobile        frequency : %0.4652136841632885
technology    frequency : %0.3934122152249031



<h1 style="color:#694382; background: #ECCFFF"> Analysis Of Part 1 </h1>
As we can see, these words are much more useful for specifying the category of text samples since these words are much more meaningful for us. For example, in entertainment category, the meaningful words that appear most are "film", "best" and "music". It is feasible since that these words are relevant to the entertainment category.

<h1 style="font-size: 40px; font-weight: 600;text-transform:uppercase; color:#256029; background:#C8E6C9"> Part 2:  Implementing Naive Bayes </h1>

<h3 style="font-weight: 600;text-transform:uppercase; color:#c63737 ; background:#FFCDD2"> Problem Definition,Solution Steps and Implementation</h3>
<ul>
        <li><span style="color:#c63737 ;">Problem Definition </span>We will implement naive bayes model for classifying the categories of text samples. We will use Bag of Words (BoW) model which learns a vocabulary from all of the documents, then models each document by counting the number of times each word appears.
  
</li>  
    <li><span style="color:#c63737 ; ">Solution Steps and Implementation : </span> 
        Bayes theorem for our case is as follows:
        <img src="https://i.ibb.co/Y31Pzdc/bayes-Theoremforass.jpg" alt="bayes-Theoremforass" border="0">
        In the formula above, we will ignore P(text) part since it is appearing in every category. 
    Here our categories are sport, business, politics, entertainment and tech.<br> We will calculate each categories probabilities and predict according to their probabilities' closeseness of zero. Naive Bayes assumes that every word in the sentence is independent of the other words. So, we can look individual words probabilities in text like below:<br>
    P( This is sport text |  category) = P(This |  category)P(is |  category)P(sport |  category)P(text |  category). <br><br>
    However, sometimes word in the test data may not appear in train data. In this case, probability would be zero. To overcome this issue, we will use laplace smoothing. We will add 1 to dominator and length of words that appeared in training data to the denominator. In this case, our probability will never be 0 so that we won't have any 0 probaility values for each category.<br><br>
     While classifying category, we are multiplying several probabilities together. This may cause to underflow since we are multiplying several small probabilities together. To solve this, we can take the logarithm of these probabilities. Since logarithms of the probabilities will have negative values, lower probabilities are further away from zero and larger probabilities are closer to zero. So we can still get the maximum of them. Also, since we take the logarithm of probabilities, we need to add them up instead of multiplying.
    </li> 
</ul>

In [113]:
#method for obtaining train and text word matrixes and also vocabulary list
def get_CountVectorizer_results(x_train, x_test, ngram_range, stopwords=frozenset()):
    #initilazing vectorizer
    vect = CountVectorizer(ngram_range=ngram_range,stop_words=stopwords)
    
    #train and text vectors
    train_cv = vect.fit_transform(x_train)
    vocabulary_list = vect.get_feature_names_out()
    test_cv = vect.transform(x_test)
    
    #train and test matrixes
    train_words_matrix = train_cv.toarray()
    test_words_matrix = test_cv.toarray()
    
    #return values
    return train_words_matrix, test_words_matrix, vocabulary_list

#fitting train data for later predicting phase
def naive_bayes_fit(x_train, y_train, train_words_matrix, vocabulary_list, labels):
    wordCountsOfLabels = {}  #each word counts in each category
    total_counts_of_labels = {}  #total numbers of each category 
    #each categorys probabilities as log value
    logs_of_each_labels = {}    #basically P(category)
    
    for l in labels:
        #defaultdict used for hadling unseen word in test matrix 
        wordCountsOfLabels[l] = defaultdict(lambda : 0)
        total_counts_of_labels[l] = len(x_train[np.where(y_train==l)])
        logs_of_each_labels[l] = math.log(total_counts_of_labels[l] / len(y_train)) #P(category)=category/total
    
    #set word counts in each category for wordCountsOfLabels
    #for each sample in training 
    for i in range(len(y_train)):
        l=y_train[i] #category of text
        #words that appeared in train sample
        avaliable_words = np.array(np.where(train_words_matrix[i]!=0))[0]
        for j in avaliable_words:
            wordCountsOfLabels[l][vocabulary_list[j]] += train_words_matrix[i][j]
    
    #return values    
    return wordCountsOfLabels, total_counts_of_labels, logs_of_each_labels

def naive_bayes_predict(test_words_matrix, wordCountsOfLabels, total_counts_of_labels, logs_of_each_labels,
                        labels , vocabulary_list):
    
    preds = [] #predictions
    #for each test sample
    for i in range(test_words_matrix.shape[0]):
        #each categories probabilty values
        label_scores = {}
        #initialize each categories probabilty values with P(category)
        for l in logs_of_each_labels:
            label_scores[l] = logs_of_each_labels[l]
        
        #words in test sample
        words_arr = []
        for j in np.where(test_words_matrix[i]!=0)[0]:
            words_arr.append(vocabulary_list[j])
        
        #for each word in test sample
        for word in words_arr:
            #for each category
            for l in labels:
                #laplace smoothing used for 0 probability , P(text|category)
                #dominator = count of word in this category
                #denominator=total count of category
                dominator = wordCountsOfLabels[l][word] + 1 #1 is laplace smoothing 
                denominator = total_counts_of_labels[l] + len(vocabulary_list) #len(vocabulary_list) is laplace smoothing
                #add to the label scores, we are adding because we use log probability
                label_scores[l] += math.log(dominator/denominator)
        #get category that has probability value closes to 0
        preds.append(max(label_scores, key=label_scores.get))
    return preds

In [114]:
#training and predicting-unigram- Stopwords not  used- TD-IDF not used
st = time.time()
train_words_matrix, test_words_matrix, vocabulary_list = get_CountVectorizer_results(X_train, X_test, ngrams["unigram"])

wordCountsOfLabels, total_counts_of_labels, logs_of_each_labels = naive_bayes_fit(X_train, y_train,
                                                                                train_words_matrix, vocabulary_list,
                                                                               label_values)

y_preds_unigram_part2 = naive_bayes_predict(test_words_matrix, wordCountsOfLabels, total_counts_of_labels, logs_of_each_labels,
                        label_values , vocabulary_list)

print("Training and Prediction Time : {} sn".format((time.time()-st)))

Training and Prediction Time : 1.2262709140777588 sn


In [115]:
#training and predicting-bigram- Stopwords not  used- TD-IDF not used
st = time.time()
train_words_matrix, test_words_matrix, vocabulary_list = get_CountVectorizer_results(X_train, X_test, ngrams["bigram"])

wordCountsOfLabels, total_counts_of_labels, logs_of_each_labels = naive_bayes_fit(X_train, y_train,
                                                                                train_words_matrix, vocabulary_list,
                                                                                label_values)

y_preds_bigram_part2=naive_bayes_predict(test_words_matrix, wordCountsOfLabels, total_counts_of_labels, logs_of_each_labels,
                        label_values , vocabulary_list)

print("Training and Prediction Time : {} sn".format((time.time()-st)))

Training and Prediction Time : 2.569582223892212 sn


<h1 style="color:#694382; background: #ECCFFF"> Analysis Of Part 2 </h1>
<ul>
    <li>Since we used Bag of Words (BoW) model, our train and text matrix dimensions are quite large. Also when training bigram model, since we use every pair of words, our matrix is much large compared to unigram. This causes that our model training quite good in unigram and a bit slow in bigram.</li>
</ul>

<h1 style="font-size: 40px; font-weight: 600;text-transform:uppercase; color:#256029; background:#C8E6C9"> Part 3:  Analyzing effect of the words on prediction and Stopwords </h1>
<h1 style=" color:#c63737 ; background:#FFCDD2"> Part 3-a: Analyzing effect of the words on prediction </h1>

In [182]:
def get_N_TfidfTransformer_values(df,category,n=10,ngram_range=(1,1), stopwords=frozenset()):
    #get the category dataset
    data = df[df["Category"]==category]
    data = data["Text"]
    #count vectorizer
    vect = CountVectorizer(ngram_range=ngram_range, stop_words=stopwords)
    data_cv = vect.fit_transform(data)
    vocabulary = vect.get_feature_names_out()
    #TfidfTransformer
    tdif_Transformer = TfidfTransformer()
    #passsing countvectorizer data array to TfidfTransformer
    tdif_Transformer.fit_transform(data_cv.toarray())
    #sort idf_ array
    sorted_tdif = np.argsort(tdif_Transformer.idf_ )
    #return last 10 elements of idf_ as presence and first 10 elements as absence                         
    return [vocabulary[i] for i in sorted_tdif][-n:], [vocabulary[i] for i in sorted_tdif][:n] 

In [183]:
#List the 10 words whose presence and absence most strongly predicts that the article
#belongs to specific category for each five categories -unigram
n = 10
presencesUnigram=[]
absencesUnigram=[]
presencesBigram=[]
absencesBigram=[]
for l in label_values:
    #presences and absences of category for unigram
    presenceUn, absenceUn = get_N_TfidfTransformer_values(df, l, n, ngrams["unigram"])
    #presences and absences of category for bigram
    presenceBi, absenceBi = get_N_TfidfTransformer_values(df, l, n, ngrams["bigram"])
    presencesUnigram.append(presenceUn)
    absencesUnigram.append(absenceUn)
    presencesBigram.append(presenceBi)
    absencesBigram.append(absenceBi)
    

<h1  style="color:#805b36 ;background:#FFD8B2"> List the 10 words whose presence most strongly predicts that the article belongs to specific category for each five categories</h1>

In [184]:
pd.DataFrame(presencesUnigram, columns=[i for i in range(1,11)], index=label_values)

,1,2,3,4,5,6,7,8,9,10
business,decelerate,orange,deceleration,opts,decentralise,decisiveness,decker,declaration,orchestrated,invests
sport,hurdlers,hurdler,hurdle,hunt,hungry,hunger,hung,hundredth,hurry,karl
politics,hutus,hutu,hussein,hurts,hurting,hurt,hurriedly,hurdles,horror,zone
entertainment,merengue,chieko,merchants,merchandise,mention,mental,mendelsohn,menagerie,cheshire,zutons
tech,indicted,indicator,indicative,indicating,indexed,independently,indelible,indefinitely,incumbent,jordanian


In [185]:
pd.DataFrame(presencesBigram, columns=[i for i in range(1,11)], index=label_values)

,1,2,3,4,5,6,7,8,9,10
business,freezing tariffs,freezing temperatures,freight capacity,freight carrying,french car,french carmaker,french champagne,french designer,freeze offered,zurich and
sport,had match,had me,had miserable,had most,had mountain,had nine,had number,had on,had lied,zvonareva while
politics,happened was,happening even,happening round,happening to,happens and,happens both,happens is,happens it,happened she,zone the
entertainment,girl was,girl with,girlfriend for,girlfriend motley,girlfriend or,girlfriends at,girls aloud,girls and,gill champion,zutons at
tech,homes is,homes that,homes this,homes will,homework but,honda and,honda asimo,honda have,homes for,many elections


<h1  style="color:#805b36 ;background:#FFD8B2"> List the 10 words whose absence most strongly predicts that the article belongs to specific category for each five categories</h1>

In [186]:
pd.DataFrame(absencesUnigram, columns=[i for i in range(1,11)], index=label_values)

,1,2,3,4,5,6,7,8,9,10
business,the,to,of,in,and,said,for,that,it,is
sport,the,to,of,in,and,for,but,on,at,with
politics,to,in,the,of,and,on,is,for,be,said
entertainment,the,in,of,and,to,on,for,was,has,with
tech,to,the,and,of,in,that,it,is,for,be


In [187]:
pd.DataFrame(absencesBigram, columns=[i for i in range(1,11)], index=label_values)

,1,2,3,4,5,6,7,8,9,10
business,in the,of the,to the,for the,on the,that the,said the,and the,the us,by the
sport,in the,of the,for the,at the,to the,on the,to be,he said,with the,year old
politics,of the,in the,to the,said the,he said,on the,to be,for the,and the,would be
entertainment,of the,in the,at the,to the,on the,for the,to be,and the,has been,with the
tech,of the,in the,to be,to the,on the,for the,that the,will be,it is,and the


<h1 style="color:#694382; background: #ECCFFF"> Analysis Of Part 3-a </h1>
<ul>
    <li>As we can see above, the words whose presence most strongly predicts the correct article category are related to that category. These words are not stop words. Because, in the most of the articles, stop words occur many times. So, the presence of the stop words actually does not predict the correct article category. But, these related words provide a better prediction.</li>
     <li>However, the words whose absence most strongly predicts the correct article category are mostly stop words, such as "to", "the", "and", "of", "in", "is" etc. One of the reason for these words are stop words is that these stop words occur many times in articles. This causes misclassification. The Naive Bayes Model can mispredict some instances because these words are in all categories. So, the absence of these words prevents misclassification and the other words in articles are more related to that relevant category.</li>
</ul>
<h1 style="color:#c63737 ; background:#FFCDD2"> Reimplementation Of Naive Bayes using TF-IDF </h1>
Since we will use TF-IDF, we need to use TfidfTransformer and its result matrixes to train our data. So we will reimplement preprocess part of our Naive Bayes and then we can do the same process in training and predicting parts like we did in part 2. Reimplementation of Naive bayes is actually instead uf using CountVectorizer matrixes , we will do the TfidfTransformer to the resulting matrixes of CountVectorizer and use the TfidfTransformer matrixes for training and testing.

In [129]:
def get_CountVectorizer_results_using_TfidfTransformer(x_train, x_test, ngram_range, stopwords=frozenset()):
    #count vectorizer results
    vect = CountVectorizer(ngram_range=ngram_range, stop_words=stopwords)
    train_cv = vect.fit_transform(x_train)
    test_cv = vect.transform(x_test)
    
    tfidf_Transformer = TfidfTransformer()
    #train and test matrixes using TfidfTransformer with CountVectorizer result matrixes
    train_fidf_cv = tfidf_Transformer.fit_transform(train_cv.toarray())
    vocabulary_list = tfidf_Transformer.get_feature_names_out()
    test_fidf_cv = tfidf_Transformer.transform(test_cv.toarray())
    
    train_words_matrix = train_fidf_cv.toarray()
    test_words_matrix = test_fidf_cv.toarray()
    
    #return train matrix ,test matrix and vocabulary list
    return train_words_matrix, test_words_matrix, vocabulary_list

In [130]:
st = time.time()

train_words_matrix, test_words_matrix, vocabulary_list = get_CountVectorizer_results_using_TfidfTransformer(X_train, X_test, 
                                                                                       ngrams["unigram"])

wordCountsOfLabels, total_counts_of_labels, logs_of_each_labels = naive_bayes_fit(X_train, y_train,
                                                                                train_words_matrix, vocabulary_list,
                                                                                label_values)

y_preds_unigram_part3_a = naive_bayes_predict(test_words_matrix, wordCountsOfLabels, total_counts_of_labels, logs_of_each_labels,
                        label_values , vocabulary_list)

print("Training and Prediction Time : {} sn".format((time.time()-st)))

Training and Prediction Time : 1.7061960697174072 sn


In [131]:
st = time.time()
train_words_matrix, test_words_matrix, vocabulary_list = get_CountVectorizer_results_using_TfidfTransformer(X_train, X_test, 
                                                                                       ngrams["bigram"])

wordCountsOfLabels, total_counts_of_labels, logs_of_each_labels = naive_bayes_fit(X_train, y_train,
                                                                                train_words_matrix, vocabulary_list,
                                                                                label_values)

y_preds_bigram_part3_a = naive_bayes_predict(test_words_matrix, wordCountsOfLabels, total_counts_of_labels, logs_of_each_labels,
                        label_values , vocabulary_list)

print("Training and Prediction Time : {} sn".format((time.time()-st)))

Training and Prediction Time : 6.50318169593811 sn


<h1 style=" color:#c63737 ; background:#FFCDD2"> Part 3-b: Stopwords </h1>

In [188]:
#List the 10 words whose presence and absence most strongly predicts that the article
#belongs to specific category for each five categories. Stopwords are removed.
n = 10
presencesUnigramStopwords=[]
absencesUnigramStopwords=[]
presencesBigramStopwords=[]
absencesBigramStopwords=[]
for l in label_values:
    #presences and absences of category for unigram
    presenceUn, absenceUn = get_N_TfidfTransformer_values(df, l, n, ngrams["unigram"], stopwords=ENGLISH_STOP_WORDS)
    #presences and absences of category for bigram
    presenceBi, absenceBi = get_N_TfidfTransformer_values(df, l, n, ngrams["bigram"], stopwords=ENGLISH_STOP_WORDS)
    presencesUnigramStopwords.append(presenceUn)
    absencesUnigramStopwords.append(absenceUn)
    presencesBigramStopwords.append(presenceBi)
    absencesBigramStopwords.append(absenceBi)
    

<h1  style="color:#805b36 ;background:#FFD8B2"> List the 10 words whose presence most strongly predicts that the article belongs to specific category for each five categories -Stopwords are Removed</h1>

In [189]:
pd.DataFrame(presencesUnigramStopwords, columns=[i for i in range(1,11)], index=label_values)

,1,2,3,4,5,6,7,8,9,10
business,cynthia,cyprus,offically,dai,offerings,daimerchrysler,daimlerchrylser,dainik,officers,intifada
sport,parling,parlour,dorival,doriva,parnaby,parr,dope,dons,pardew,00
politics,humiliation,humiliating,humberside,humankind,humanitarian,hulme,hug,hrh,humour,zone
entertainment,heir,heels,hecuba,hectoring,hectic,hecklers,heavyweights,heavier,hellboy,zutons
tech,einar,educations,eff,efficacy,pipex,pipes,pipeline,efficiencies,eeoc,00


In [190]:
pd.DataFrame(presencesBigramStopwords, columns=[i for i in range(1,11)], index=label_values)

,1,2,3,4,5,6,7,8,9,10
business,expand robustly,expanded 15,expanded 2004,expanded 25,expanded annual,expanded breakneck,expanded past,expanded service,expand production,zurich london
sport,game hot,game ideal,game igor,game illusions,game impartial,game insisted,game intention,game involved,game hosts,zvonareva wimbledon
politics,force used,force world,forced 1967,forced apologise,forced bat,forced change,forced decide,forced deny,force tuesday,zone scottish
entertainment,film overall,film new,film official,film opened,film opening,film organisers,film original,film originally,film nominations,zutons 20
tech,labs needed,labs set,labs shut,labs synergies,cafe opening,lack confidence,lack dvd,cafe killed,cafes 200,00 minute


<h1  style="color:#805b36 ;background:#FFD8B2"> List the 10 words whose absence most strongly predicts that the article belongs to specific category for each five categories -Stopwords are Removed</h1>

In [193]:
pd.DataFrame(absencesUnigramStopwords, columns=[i for i in range(1,11)], index=label_values)

,1,2,3,4,5,6,7,8,9,10
business,said,year,new,market,company,firm,mr,chief,2004,years
sport,said,year,time,win,game,world,just,old,won,team
politics,said,mr,government,people,minister,labour,election,new,party,told
entertainment,said,year,film,new,best,star,including,time,uk,years
tech,said,people,new,use,year,technology,make,way,mr,like


In [192]:
pd.DataFrame(absencesBigramStopwords, columns=[i for i in range(1,11)], index=label_values)

,1,2,3,4,5,6,7,8,9,10
business,chief executive,analysts said,economic growth,stock market,new york,oil prices,said mr,said statement,stock exchange,news agency
sport,year old,told bbc,grand slam,bbc sport,second half,world cup,australian open,manchester united,champions league,fly half
politics,tony blair,prime minister,general election,mr blair,told bbc,michael howard,said mr,liberal democrats,gordon brown,lib dems
entertainment,year old,new york,los angeles,box office,best film,won best,best director,best actress,named best,film festival
tech,said mr,bbc news,told bbc,news website,mobile phone,mobile phones,let people,high speed,digital music,consumer electronics


<h1 style="color:#694382; background: #ECCFFF"> Analysis Of Part 3-b </h1>
<ul>
    <li>In this part, we remove the stop words from dictionary and then run the naive bayes algorithm. When we take a look above, we can easily say that most of the non-stop words whose presence most strongly predicts the correct article category are relevant to predicted category. Also, there are not many of these words in other categories. Therefore, these relevant non-stop words provide a better prediction and increase the accuracy of the model.</li>
     <li>The words whose absence most strongly predicts the correct article will no longer be stop words because stop words are removed. Stop words are insignificant for classifciation. So, absence of these non-stop words provides a better prediction.</li>
</ul>

<h1 style="color:#c63737 ; background:#FFCDD2"> Running Naive Bayes with English Stop Words Removed</h1>

In [122]:
st = time.time()
train_words_matrix, test_words_matrix, vocabulary_list = get_CountVectorizer_results(X_train, X_test, 
                                                                                       ngrams["unigram"],
                                                                                  stopwords=ENGLISH_STOP_WORDS)

wordCountsOfLabels, total_counts_of_labels, logs_of_each_labels = naive_bayes_fit(X_train, y_train,
                                                                                train_words_matrix, vocabulary_list,
                                                                                label_values)

y_preds_unigram_part3_b_stopwords = naive_bayes_predict(test_words_matrix, wordCountsOfLabels, total_counts_of_labels, logs_of_each_labels,
                        label_values , vocabulary_list)

print("Training and Prediction Time : {} sn".format((time.time()-st)))

Training and Prediction Time : 0.999222993850708 sn


In [123]:
st = time.time()
train_words_matrix, test_words_matrix, vocabulary_list = get_CountVectorizer_results(X_train, X_test, 
                                                                                       ngrams["bigram"],
                                                                                  stopwords=ENGLISH_STOP_WORDS)

wordCountsOfLabels, total_counts_of_labels, logs_of_each_labels = naive_bayes_fit(X_train, y_train,
                                                                                train_words_matrix, vocabulary_list,
                                                                                label_values)

y_preds_bigram_part3_b_stopwords = naive_bayes_predict(test_words_matrix, wordCountsOfLabels, total_counts_of_labels, logs_of_each_labels,
                        label_values , vocabulary_list)

print("Training and Prediction Time : {} sn".format((time.time()-st)))

Training and Prediction Time : 1.742386817932129 sn


<h1 style="font-size: 40px; font-weight: 600;text-transform:uppercase; color:#256029; background:#C8E6C9"> Part 4:   Calculation of Accuracy </h1>

In [124]:
print("Part 2 Results : ")
print("Unigram Accuracy - TF-IDF is not used : ", accuracy_score(y_test, y_preds_unigram_part2))
print("Bigram Accuracy - TF-IDF is not used : ", accuracy_score(y_test, y_preds_bigram_part2))

Part 2 Results : 
Unigram Accuracy - TF-IDF is not used :  0.9194630872483222
Bigram Accuracy - TF-IDF is not used :  0.930648769574944


In [125]:
print("Part 3-a Results : ")
print("Unigram Accuracy - TF-IDF used :", accuracy_score(y_test,y_preds_unigram_part3_a))
print("Bigram Accuracy - TF-IDF used : ", accuracy_score(y_test,y_preds_bigram_part3_a))

Part 3-a Results : 
Unigram Accuracy - TF-IDF used : 0.9172259507829977
Bigram Accuracy - TF-IDF used :  0.9284116331096197


In [126]:
print("Part 3-b Results : ")
print("Unigram Accuracy - Stopwords are removed: ", accuracy_score(y_test,y_preds_unigram_part3_b_stopwords))
print("Bigram Accuracy - Stopwords are removed : ", accuracy_score(y_test,y_preds_bigram_part3_b_stopwords))


Part 3-b Results : 
Unigram Accuracy - Stopwords are removed:  0.959731543624161
Bigram Accuracy - Stopwords are removed :  0.9686800894854586


<h1 style="color:#694382; background: #ECCFFF"> Analysis Of Part 4 </h1>
<ul>
    <li>When we look at the results, the performance of the model is very good. If we do not use TF-IDF algorithm and not remove stopwords, accuracy of the model will be above 90%. Also, accuracy of the bigram BoW model is higher than accuracy of the unigram BoW model. One reason for this difference is that there are many phrases in the English language. A word can be related to the previous or the next word.
    </li>
    <li>
        When we remove stop words, the performance of the model increases. The accuracy of the model is higher than 95%. The presence of the stop words cause misclasification. Because, stop words such as "the", "in", "is", "of" etc. are in all of the articles many times. When we test a new article which also has many stop words, the probability of these stop words is higher than most of the other words. So, this makes the classification difficult. However, if we remove stop words from the dictionary, this problem will no longer be a problem.
    </li>   
</ul>